In [1]:
import pandas as pd
import numpy as np
import replicaEVSE.load_curve as sim
import replicaEVSE.datautils as simdu
import matplotlib.pyplot as plt
import os
import joblib
from tqdm import tqdm
from glob import glob
# import dask.dataframe as dd
import pandas as dd
import datetime as dt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%reload_ext autoreload
%autoreload 2

datadir = '../../data/'

 1) read in the loads and charges
 2) compute the peak load
 3) compute the fraction of charging events as a function of charger type

In [2]:
seg_list = ['Personal Sedan',
 'Personal Crossover',
 'Personal Truck/SUV',
 'Commercial Sedan',
 'Commercial Crossover',
 'Commercial Truck/SUV']

# read in the loads and charges

In [52]:
charge_files = glob(datadir + 'loads_charges/charges_2*_2023-06-06.parquet')
charges_list = []
for cfile in charge_files:
    charges = dd.read_parquet(cfile)
    # make the files a bit smaller
    charges[charges['charger_power_kW'] > 0]
    charges_list.append(charges)
charges = dd.concat(charges_list)

loads_files = glob(datadir + 'loads_charges/loads_2*_2023-06-06.parquet')
loads_list = []
for lfile in loads_files:
    loads = dd.read_parquet(lfile)
    loads_list.append(loads)
loads = dd.concat(loads_list)


In [53]:
charges.columns, loads.columns

(Index(['person_id', 'charge_id', 'charge_type', 'activity_id', 'simulation_id',
        'charger_power_kW', 'charge_energy_used_kWh',
        'charge_opportunity_remaining_kWh', 'weekday', 'year', 'segment',
        'work_frac', 'multiunit_frac'],
       dtype='object'),
 Index(['person_id', 'load_segment_id', 'charge_id', 'charge_type',
        'window_start_time', 'window_end_time', 'load_kW', 'weekday', 'year',
        'segment', 'work_frac', 'multiunit_frac'],
       dtype='object'))

In [54]:
reftime = ref_time=dt.datetime(2023, 1, 1, 0, 0)
loads["hour"] = (loads["window_end_time"]+ref_time).dt.hour

### Join the load and charges tables

In [56]:
loadsmall = loads[['charge_id', 'hour', 'load_kW']]
loadsmall = loadsmall[loadsmall['load_kW'] > 0] # remove 0 loads
all = loadsmall.merge(charges, on='charge_id', how='left')

### Find the max load per year and fraction of that load by charger type.

In [58]:
max_load = all.groupby(['year','hour',])['load_kW'].sum()

We can edit the granularity of our charging events.

In [59]:
df = all.copy()
df['location'] = df.charge_type.copy()
keep_values = ['single_family_home', 'multi_family_home', 'public']
home_values = ['public', 'work']

# Set values not in the keep list to 'work'
df['location'] = df['location'].where(df['location'].isin(keep_values), 'work')

In [78]:
def calc_load_frac_by_location(df):
    load_by_loc = df.groupby(['year','location', 'hour', ])['load_kW'].sum()
    load_by_loc = load_by_loc.groupby(['year', 'location']).max().reset_index()

    totload = load_by_loc.groupby('year')['load_kW'].sum().reset_index()
    totload['tot_load_kW'] = totload['load_kW']
    totload.drop(columns=['load_kW'], inplace=True)

    # merge the total load back into the load_by_loc dataframe
    merged = pd.merge(load_by_loc, totload, on='year', how='left')
    merged['load_frac'] = merged['load_kW'] / merged['tot_load_kW']
    merged.to_csv(datadir + 'loads_charges/frac_load_by_loc.csv', index=False)
    return merged

merged = calc_load_frac_by_location(df)
merged

,year,location,load_kW,tot_load_kW,load_frac
0,2023,multi_family_home,1.558815e+04,7.943673e+05,0.019623
1,2023,public,2.208862e+05,7.943673e+05,0.278066
2,2023,single_family_home,5.491770e+05,7.943673e+05,0.691339
3,2023,work,8.715921e+03,7.943673e+05,0.010972
4,2025,multi_family_home,6.167131e+04,1.772348e+06,0.034796
5,2025,public,4.708865e+05,1.772348e+06,0.265685
6,2025,single_family_home,1.212835e+06,1.772348e+06,0.684310
7,2025,work,2.695521e+04,1.772348e+06,0.015209
8,2030,multi_family_home,1.845952e+05,3.547069e+06,0.052042
9,2030,public,9.007578e+05,3.547069e+06,0.253944


# Summary statistics for use of charging